In [1]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch
from tqdm import tqdm  # 진행률 표시를 위한 tqdm 라이브러리

mirrored_strategy = tf.distribute.MirroredStrategy()
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

2024-04-23 01:32:24.183270: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-23 01:32:24.227936: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 01:32:24.986562: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/undergrad/miniconda3/envs/tf-2.12.0/lib/python3.8/site-packages/huggingface_hub/utils/_runtime.py:185: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydant

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


2024-04-23 01:32:28.986159: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9630 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3b:00.0, compute capability: 7.5
2024-04-23 01:32:28.986793: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9630 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:af:00.0, compute capability: 7.5
2024-04-23 01:32:28.987319: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9630 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:d8:00.0, compute capability: 7.5
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_

In [2]:
filename = 'prompt.txt'
f = open(filename, 'r')
lines = f.readlines()
print(lines)
print('-' * 10)

['아침 일과 시작하기\n', '책 읽기의 즐거움\n', '가족과 함께 하는 저녁 식사\n', '친구와의 영화 관람\n', '주말 동안의 여행 계획\n', '정원 가꾸기의 효과\n', '요리를 통한 스트레스 해소\n', '자전거 타기의 장점\n', '집안 정리 정돈 팁\n', '개인 운동 루틴 유지하기\n', '취미 활동의 중요성\n', '새로운 언어 배우기\n', '음악 연주와 그 영향\n', '사진 촬영을 통한 추억 만들기\n', '지역 사회 봉사의 보람\n', '패션과 개인 스타일 표현\n', '휴일을 활용한 가족 모임\n', '건강한 식생활을 위한 조리법\n', '애완동물과 보내는 시간\n', '독서 클럽에서의 토론\n', '도서관 방문의 장점\n', '집에서의 필름 감상\n', '커피 한 잔과의 여유\n', '일기 쓰기의 치유 효과\n', '홈 베이킹의 즐거움\n', '직접 만든 공예품\n', '새로운 요리법 시도하기\n', '지역 공연 관람\n', '자연 속에서의 하이킹\n', '가구 리폼 프로젝트\n', '아이와 함께하는 공원 나들이\n', '새로운 스포츠 도전\n', '개인 기록의 개선\n', '집에서의 요가 세션\n', '반려동물과의 산책\n', '가족 앨범 만들기\n', '매일의 명상 시간\n', '가벼운 아침 조깅\n', '주말 브런치 준비\n', '일상의 소소한 행복 찾기\n', '스마트폰 없는 하루\n', '책상 정리의 중요성\n', '옷장 정리하기\n', '새벽 시간의 고요함\n', '오후의 티타임\n', '자기 전 명상\n', '집안 식물 가꾸기\n', '근처 박물관 탐방\n', '집에서의 칵테일 파티\n', '커뮤니티 클래스 참여\n', '셀프 케어 루틴\n', '장거리 걷기의 이점\n', '텃밭 가꾸기\n', '개인적인 스타일 개발\n', '책 추천 공유\n', '로컬 마켓 쇼핑\n', '장기 게임의 즐거움\n', '가족과의 보드게임 밤\n', '디지털 디톡스\n', '가까운 도시 탐험\n', '수제 비누 만들기\n', '로컬 아

In [5]:

with open('perplexity_g=0.5_d=40.0.txt', 'w') as f:  #file내용 clear
    f.write('')
    
with open('output_g=0.5_d=40.0.txt', 'w') as f:  #file내용 clear
    f.write('')

for line in lines:
    line = line.strip()
    print(f"prompt : {line}")

    input_text = line
    input_ids = tokenizer.encode(input_text, return_tensors='tf')
    #term = 3

    last_index = input_ids.shape[-1]
    nlls = []
            
    with mirrored_strategy.scope():
        for _ in tqdm(range(100)):  # 최대 100번 반복
            if input_ids.shape[-1] >= 100:
                break  # input_ids의 길이가 100 이상이면 반복 중단
            
            #print("================================")
            #print(f"문장의 토큰 개수 : {input_ids.shape[-1]}")
            #set 나눠서 green set 로짓을 촉진하기
            gamma = 0.5
            delta = 40.0
            Logits = model(input_ids).logits
            #print(f"original Logits : {Logits}")
            
            #seed 생성
            seed = input_ids[-1] % Logits.shape[-1]
            #print(f"seed : {seed[-1]}")
            np.random.seed(seed[-1])

            #random green list 설정
            indices = np.arange(Logits.shape[-1]) #0~51999
            green_list_size = int(Logits.shape[-1] * gamma)  #25600
            green_list = np.random.choice(indices, green_list_size, replace=False)
            #print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

            #--여기부터 probability 구하기
            # green logit과 red logit의 값을 담을 리스트 생성
            green_logits = tf.zeros(green_list_size)
            red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

            #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
            Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
            Logits_num = Logits.numpy()
            for i in green_list :
                Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
            Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
            #print(f"updated Logits : {Logits}")
            
            # softmax 계산을 위한 분모
            denominator = tf.reduce_sum(tf.exp(Logits), axis=-1)  
            #print(f"denominator : {denominator}") #25600개의 green list

            # Logits 텐서에 softmax 적용하여 확률 계산 (병렬 계산), softmax_tensor = probability
            softmax_tensor = Logits.numpy()
            softmax_tensor[0, last_index-1, :] = np.exp(softmax_tensor[0, last_index-1, :])
            softmax_tensor[0, last_index-1, :] /= denominator[0,last_index-1]
            #print(softmax_tensor.shape)
            #print(f"Probabilities : {softmax_tensor}")

            # 데이터 타입 확인 및 조정
            add_token_id = tf.argmax(softmax_tensor[0][last_index-1])
            #print(f"updated Logits Max value : {add_token_id}")
            
            # Calculate perplexity
            # 참고 https://huggingface.co/docs/transformers/main/ko/perplexity
            selected_probability = softmax_tensor[0, last_index-1, add_token_id]
            nlls.append(np.log(selected_probability))
            
            #print(softmax_tensor[0, last_index-1, add_token_id])

            # 텐서 연결
            add_token_id = tf.reshape(add_token_id, [1,1])
            add_token_id = tf.cast(add_token_id, input_ids.dtype)
            input_ids = tf.concat([input_ids, add_token_id], axis=-1)
            next_token = tokenizer.decode(input_ids.numpy()[0,:])
            #print(f"next token : {next_token}")
            
            last_index = input_ids.shape[-1]
            #term = 3
        final_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(final_token)

        # Calculate perplexity
        perplexity = tf.exp(-tf.reduce_mean(nlls))
        print("Perplexity: ", perplexity.numpy())
        
        perplexity_str = str(perplexity.numpy())  # perplexity를 문자열로 변환
        with open('perplexity_g=0.5_d=40.0.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
            f.write(perplexity_str + '\n')
        with open('output_g=0.5_d=40.0.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
            f.write(final_token + '\n')

prompt : 아침 일과 시작하기


 96%|█████████▌| 96/100 [00:16<00:00,  5.68it/s]


아침 일과 시작하기 전에 미리 준비해두면 좋겠습니다.
오늘은 고기압 가장자리에 들어 가끔 맑은 날씨가 이어지겠지만, 제주와 남해안, 동해안은 동풍이 불면서 낮 동안 가끔 비가 조금 내리겠습니다.
내일 아침기온 서울 7도, 대전 7도, 대구 9도로 시작하겠습니다.
내일 낮기온 서울 11도, 대전 11도, 대구 12도 등으로 평년에 비해 높습니다.
바다 물결은 동해 먼바다가 최고 2.5m까지 높게, 남해 서부앞바다와 동해 먼바다가 최고 2.5
Perplexity:  2.0952396
prompt : 책 읽기의 즐거움


 95%|█████████▌| 95/100 [00:16<00:00,  5.78it/s]


책 읽기의 즐거움도 함께 느낄 것이다.
이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다.
이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '
Perplexity:  2.8152173
prompt : 가족과 함께 하는 저녁 식사


...

지역사회와의 소통 강화 및 지역경제 활성화를 위해 노력하겠다”며 “이러할 경우 2017년부터 2020년까지 총 4000억원 이상을 투자해 일자리 1만개를 만들겠다. 지역경제 활성화를 위해 노력하겠다”는 의지를 밝혔다.
김 시장은 또 “지난해 7월 광주시가 광주전남연구원과 함께 개최한 '광주형 일자리 모델 구축 및 활성화 전략 세미나'에 참석한 것을 계기로 광주시와 광주전남연구원이 공동으로 일자리 모델을 구축하는 데 협력하기로 합의했다. 광주시는 이를 통해 일자리 정책 및
Perplexity:  3.6734626
prompt : 가족과의 소중한 대화


 96%|█████████▌| 96/100 [00:16<00:00,  5.96it/s]


가족과의 소중한 대화에서 '진심 어린 사과'를 해야 한다"고 말했다.
이어 "국민이 납득과 이해를 할 수 있는 사과문을 써야 한다"며 "이번 사과는 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자, 국민과의 약속이자,
Perplexity:  1.8248521
prompt : 개인 발전을 위한 목표 설정


 95%|█████████▌| 95/100 [00:15<00:00,  5.94it/s]

개인 발전을 위한 목표 설정과 실행, 사업화 지원 등을 중점적으로 추진할 예정이다.
이번 협약을 통해 양사는 ▲공동연구 개발 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동연구 수행 ▲공동
Perplexity:  2.0254521


In [16]:
# 첫 번째 파일에서 숫자 읽어오기
with open('perplexity.txt', 'r') as f:
    list1 = [float(line.strip()) for line in f]

# 두 번째 파일에서 숫자 읽어오기
with open('perplexity_g=0.5_d=20.0.txt', 'r') as f:
    list2 = [float(line.strip()) for line in f]

# 두 번째 리스트에서 더 큰 값을 가진 경우를 세기 위한 카운터 초기화
count_second_list_bigger = 0
count_same = 0

# 두 리스트를 순회하면서 두 번째 리스트가 더 큰 경우를 세어줌
for val1, val2 in zip(list1, list2):
    if val2 > val1:  # 두 번째 리스트의 값이 더 큰 경우
        count_second_list_bigger += 1
    if val2 == val1 :
        count_same += 1

# 결과 출력
print("두 번째 리스트에서 더 큰 값을 가진 경우의 수:", count_second_list_bigger)
print("성능 같은 경우 :", count_same)
print("두 번째 리스트에서 더 작은 을 가진 경우의 수:", 100-count_same-count_second_list_bigger)


두 번째 리스트에서 더 큰 값을 가진 경우의 수: 36
성능 같은 경우 : 0
두 번째 리스트에서 더 작은 을 가진 경우의 수: 64


In [5]:
#delta = 1.0, gamma = 0.3 해서 비교해보기

tf.Tensor(888.30756, shape=(), dtype=float32)
